In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
from preprocessing import load_if_in_country_or_lang
from preprocessing import categorize_views_for_df
from processing import filter_by_region_cat_code, build_regression_dataframe, load_preprocessed_events
from plot_regression import plot_regression_results_from_dict, plot_regression_results_interactions_from_dict, plot_cat_by_cat_variable
import numpy as np
from regression import transform_vars_for_regression, fit_nb_with_estimated_alpha, estimate_alpha, fit_nb, standardize_var, fit_nb_with_estimated_alpha_all_codes, standardize_var

In [3]:
df_crawled = pd.read_csv('events/new/processed_0_manually_with_wikiviews.csv').drop_duplicates()

In [6]:
from processing import calculate_uniqueness, calculate_uniqueness_two_columns

# how should uniqueness decay?
df_crawled.event_date = pd.to_datetime(df_crawled.event_date)
df_crawled = calculate_uniqueness(df_crawled, 30, 'country')
df_crawled = calculate_uniqueness(df_crawled, 30, 'cat')
df_crawled = calculate_uniqueness_two_columns(df_crawled, 30, 'country', 'cat')

In [11]:
df_reg = df_crawled[(df_crawled.cat != 'undefined') & (df_crawled.event_date.dt.year >= 2016) & (df_crawled.views_7_sum > 10)].copy()
df_reg = transform_vars_for_regression(df_reg)

In [68]:
code_reg = 'C(code, Treatment(reference="en"))'
cat_reg = 'C(cat, Treatment(reference="sports"))'
class_reg = 'C(gni_class, Treatment(reference="H"))'
region_reg = 'C(gni_region, Treatment(reference="North America"))'
continent_reg = 'C(continent, Treatment(reference="North America"))'

features = [cat_reg, 'population_log', 'GDP_pc_z', code_reg, class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log']
features_code0 = [cat_reg, 'population_log', 'GDP_pc_z', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log']
features_code1 = [cat_reg, 'population_log', 'GDP_pc_z', class_reg, continent_reg, 'country_cat_articles_log', 'views_baseline_log']
features_code2 = [cat_reg, 'population_log', 'GDP_pc_z', region_reg, 'country_cat_articles_log', 'views_baseline_log']
features_code3 = [cat_reg, 'population_log', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log']

In [ ]:
'code', 'in_code_lang', 'cat', 'planned', 'surprised', 'gni_region', 'continent', 'views_baseline_log', 'population_log', 'view_country_article_log', 'country_articles_log', 'cat_articles_log', 'country_cat_articles_log', 'gni_class', 'GDP_pc_log', 'economic_region', 'oecd'

In [240]:
{"+".join(features)}

{'C(cat, Treatment(reference="sports"))+population_log+GDP_pc_z+C(gni_class, Treatment(reference="H"))+C(gni_region, Treatment(reference="North America"))+country_cat_articles_log+views_baseline_log+view_country_article_log'}

In [65]:
reg_1 = fit_nb_with_estimated_alpha(df_reg, f'views_7_sum ~ {"+".join(features)}')
reg_1.summary()

Estimated alpha as 16.06 (p=0.0000)
Deviance: 3145.17 | Null-deviance: 3871.40 | Pseudo R² (1 - D/D_0) = 0.1876
Pseudo ChiSq: 13338.80 | Good-Fit-ChiSq: 12991.57 | Residual DF: 12728
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.05


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            views_7_sum   No. Observations:                12749
Model:                            GLM   Df Residuals:                    12728
Model Family:        NegativeBinomial   Df Model:                           20
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3798e+05
Date:                Thu, 19 May 2022   Deviance:                       3145.2
Time:                        08:02:14   Pearson chi2:                 1.33e+04
No. Iterations:                   193                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    3.8786      0.690      5.624      0.000       2.527       5.230
cat[T.culture]                               0.3804      0.145      2.628      0.009       0.097       0.664
cat[T.disaster]                              0.7608      0.133      5.711      0.000       0.500       1.022
cat[T.politics]                              0.5343      0.100      5.341      0.000       0.338       0.730
code[T.de]                                  -0.4858      0.153     -3.182      0.001      -0.785      -0.187
code[T.es]                                  -0.9514      0.126     -7.538      0.000      -1.199      -0.704
code[T.it]                                  -0.9156      0.191     -4.783      0.000      -1.291      -0.540
gni_class[T.L]                               0.1457      0.361      0.404      0.686      -0.561       0.853
gni_class[T.LM]                              0.0589      0.212      0.278      0.781      -0.357       0.475
gni_class[T.UM]                              0.3480      0.149      2.331      0.020       0.055       0.641
gni_region[T.East Asia & Pacific]           -0.0474      0.163     -0.291      0.771      -0.367       0.272
gni_region[T.Europe & Central Asia]         -0.0137      0.140     -0.098      0.922      -0.288       0.260
gni_region[T.Latin America & Caribbean]      0.2908      0.200      1.456      0.145      -0.101       0.682
gni_region[T.Middle East & North Africa]     0.6824      0.250      2.727      0.006       0.192       1.173
gni_region[T.South Asia]                    -0.1929      0.313     -0.615      0.538      -0.807       0.421
gni_region[T.Sub-Saharan Africa]             0.4111      0.326      1.263      0.207      -0.227       1.049
population_log                               0.1310      0.037      3.524      0.000       0.058       0.204
GDP_pc_z                                     0.0415      0.069      0.603      0.546      -0.093       0.177
country_cat_articles_log                    -0.1585      0.038     -4.145      0.000      -0.234      -0.084
views_baseline_log                           0.0832      0.031      2.702      0.007       0.023       0.144
view_country_article_log                     0.2544      0.101      2.516      0.012       0.056       0.453
============================================================================================================
"""

In [73]:
fit_0 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_code0)}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.96 (p=0.0000)
Deviance: 1632.89 | Null-deviance: 1773.85 | Pseudo R² (1 - D/D_0) = 0.0795
Pseudo ChiSq: 6372.17 | Good-Fit-ChiSq: 6868.21 | Residual DF: 6677
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.95
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6677
Model Family:        NegativeBinomial   Df Model:                           17
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -77612.
Date:                Thu, 19 May 2022   Deviance:                       1632.

In [ ]:
reg_1 = fit_nb_with_estimated_alpha(df_reg, f'views_7_sum ~ {code_reg} * ({"+".join(features_code0)})')

In [62]:
print(features_code0)

['C(cat, Treatment(reference="sports"))', 'population_log', 'GDP_pc_z', 'C(gni_class, Treatment(reference="H"))', 'C(gni_region, Treatment(reference="North America"))', 'country_cat_articles_log', 'views_baseline_log']


In [54]:
fit_income = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_code0)}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 16.14 (p=0.0001)
Deviance: 1621.45 | Null-deviance: 1754.38 | Pseudo R² (1 - D/D_0) = 0.0758
Pseudo ChiSq: 6477.27 | Good-Fit-ChiSq: 6869.22 | Residual DF: 6678
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.97
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6678
Model Family:        NegativeBinomial   Df Model:                           16
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -77669.
Date:                Thu, 19 May 2022   Deviance:                       1621.

In [55]:
reg_2 = fit_nb_with_estimated_alpha(df_reg, f'views_7_sum ~ {code_reg} * ({"+".join(features_code2)})')

Estimated alpha as 15.69 (p=0.0000)
Deviance: 3125.90 | Null-deviance: 3962.76 | Pseudo R² (1 - D/D_0) = 0.2112
Pseudo ChiSq: 14718.85 | Good-Fit-ChiSq: 12956.21 | Residual DF: 12693
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.16


In [56]:
features_code2

['C(cat, Treatment(reference="sports"))',
 'population_log',
 'GDP_pc_z',
 'C(gni_region, Treatment(reference="North America"))',
 'country_cat_articles_log',
 'views_baseline_log']

In [57]:
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_code2)}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 16.30 (p=0.0002)
Deviance: 1613.74 | Null-deviance: 1737.15 | Pseudo R² (1 - D/D_0) = 0.0710
Pseudo ChiSq: 7091.02 | Good-Fit-ChiSq: 6872.27 | Residual DF: 6681
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.06
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6681
Model Family:        NegativeBinomial   Df Model:                           13
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -77722.
Date:                Thu, 19 May 2022   Deviance:                       1613.

In [58]:
reg_3 = fit_nb_with_estimated_alpha(df_reg, f'views_7_sum ~ {code_reg} * ({"+".join(features_code3)})')

Estimated alpha as 15.54 (p=0.0000)
Deviance: 3143.89 | Null-deviance: 4002.51 | Pseudo R² (1 - D/D_0) = 0.2145
Pseudo ChiSq: 14077.96 | Good-Fit-ChiSq: 12948.12 | Residual DF: 12685
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.11


In [59]:
fit_3 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_code3)}', 
    est_method='lbfgs', output_lvl=2)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 16.13 (p=0.0001)
Deviance: 1622.76 | Null-deviance: 1754.78 | Pseudo R² (1 - D/D_0) = 0.0752
Pseudo ChiSq: 6592.59 | Good-Fit-ChiSq: 6870.24 | Residual DF: 6679
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.99
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6679
Model Family:        NegativeBinomial   Df Model:                           15
Link Function:                    log   Scale:                          1.0000
Method:                         lbfgs   Log-Likelihood:                -77669.
Date:                Thu, 19 May 2022   Deviance:                       1622.

In [60]:
' + '.join(features_code3)

'C(cat, Treatment(reference="sports")) + population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log'

In [61]:

reg_4 = fit_nb_with_estimated_alpha(
    df_reg, f'views_7_sum ~ {code_reg} * (population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log)')

Estimated alpha as 17.57 (p=0.0000)
Deviance: 2879.58 | Null-deviance: 3539.35 | Pseudo R² (1 - D/D_0) = 0.1864
Pseudo ChiSq: 12200.92 | Good-Fit-ChiSq: 12960.25 | Residual DF: 12697
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.96


# Feature Red

In [179]:
# (2*(fit_de.df_model + 1) - 2* fit_de.llf)
#((fit_de.df_model + 1) * np.log(fit_de.nobs)) - 2* fit_de.llf
features = [cat_reg, 'population_log', 'GDP_pc_z', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log']
features_red0 = [cat_reg, 'population_log', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log']
features_red1 = [cat_reg, 'population_log', 'GDP_pc_z', region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log']
features_red2 = [cat_reg, 'population_log', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log']
features_mrmr = [cat_reg, 'population_log', 'GDP_pc_z', class_reg, region_reg, 'country_cat_articles_log', 'views_baseline_log', 'view_country_article_log', 'oecd', 'in_code_lang', 'planed']

In [108]:
reg_1 = fit_nb_with_estimated_alpha(df_reg, f'views_7_sum ~ {code_reg} * ({"+".join(features)})')

Estimated alpha as 15.54 (p=0.0000)
Deviance: 3137.94 | Null-deviance: 4001.82 | Pseudo R² (1 - D/D_0) = 0.2159
Pseudo ChiSq: 13971.94 | Good-Fit-ChiSq: 12944.08 | Residual DF: 12681
AIC: 275364.48 | BIC: -116738.19
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.10


In [175]:
fit_0 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features)}', 
    est_method='lbfgs', output_lvl=1)

________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.96 (p=0.0000)
Deviance: 1632.89 | Null-deviance: 1773.85 | Pseudo R² (1 - D/D_0) = 0.0795
Pseudo ChiSq: 6372.17 | Good-Fit-ChiSq: 6868.21 | Residual DF: 6677
AIC: 155259.62 | BIC: -57185.57
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.95
________________________________________________________________________________________________
================= Fitting es =================
Estimated alpha as 15.83 (p=0.0000)
Deviance: 650.66 | Null-deviance: 754.27 | Pseudo R² (1 - D/D_0) = 0.1374
Pseudo ChiSq: 3082.63 | Good-Fit-ChiSq: 2918.08 | Residual DF: 2794
AIC: 56449.56 | BIC: -21538.31
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.10
________________________________________________________________________________________________
================= Fitting de =========

In [174]:
print(" + ".join(features_red0))
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_red0)}', 
    est_method='lbfgs', output_lvl=1)

C(cat, Treatment(reference="sports")) + population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log + view_country_article_log
________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.98 (p=0.0000)
Deviance: 1631.06 | Null-deviance: 1771.95 | Pseudo R² (1 - D/D_0) = 0.0795
Pseudo ChiSq: 6343.44 | Good-Fit-ChiSq: 6869.22 | Residual DF: 6678
AIC: 155268.07 | BIC: -57196.22
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.95
________________________________________________________________________________________________
================= Fitting es =================
Estimated alpha as 15.84 (p=0.0000)
Deviance: 650.38 | Null-deviance: 753.94 | Pseudo R² (1 - D/D_0) = 0.1374
Pseudo ChiSq: 3087.52 | Good-Fit-ChiSq: 2919.11 | Residual DF: 2795
AIC: 56449.42 | BIC

In [158]:
print(" + ".join(features_red1))
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_red1)}', 
    est_method='lbfgs', output_lvl=2)

C(cat, Treatment(reference="sports")) + population_log + GDP_pc_z + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log + view_country_article_log
________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 16.06 (p=0.0001)
Deviance: 1629.07 | Null-deviance: 1763.22 | Pseudo R² (1 - D/D_0) = 0.0761
Pseudo ChiSq: 6592.68 | Good-Fit-ChiSq: 6871.25 | Residual DF: 6680
AIC: 155318.74 | BIC: -57215.83
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.99
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6680
Model Family:        NegativeBinomial   Df Model:                           14
Link Fun

In [162]:
print(" + ".join(features_red2))
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_red2)}', 
    est_method='lbfgs', output_lvl=2)

C(cat, Treatment(reference="sports")) + population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log
________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 16.13 (p=0.0001)
Deviance: 1622.76 | Null-deviance: 1754.78 | Pseudo R² (1 - D/D_0) = 0.0752
Pseudo ChiSq: 6592.59 | Good-Fit-ChiSq: 6870.24 | Residual DF: 6679
AIC: 155369.52 | BIC: -57213.33
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 0.99
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6679
Model Family:        NegativeBinomial   Df Model:                           15
Link 

In [166]:
print(" + ".join(features_red2))
features_comb3 = 'population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + C(cat, Treatment(reference="sports"))*country_cat_articles_log + views_baseline_log'
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {features_comb3}', 
    est_method='lbfgs', output_lvl=2)

C(cat, Treatment(reference="sports")) + population_log + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log
________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.15 (p=0.0001)
Deviance: 1708.64 | Null-deviance: 1869.16 | Pseudo R² (1 - D/D_0) = 0.0859
Pseudo ChiSq: 6708.95 | Good-Fit-ChiSq: 6867.19 | Residual DF: 6676
AIC: 154738.82 | BIC: -57101.02
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.00
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6676
Model Family:        NegativeBinomial   Df Model:                           18
Link 

In [180]:
features_mrmr

['C(cat, Treatment(reference="sports"))',
 'population_log',
 'GDP_pc_z',
 'C(gni_class, Treatment(reference="H"))',
 'C(gni_region, Treatment(reference="North America"))',
 'country_cat_articles_log',
 'views_baseline_log',
 'view_country_article_log',
 'oecd',
 'in_code_lang',
 'planed']

In [183]:
print(" + ".join(features_mrmr))
fit_2 = fit_nb_with_estimated_alpha_all_codes(['en', 'es', 'de', 'it'], df_reg, f'views_7_sum ~ {"+".join(features_mrmr)}', 
    est_method='lbfgs', output_lvl=2)

C(cat, Treatment(reference="sports")) + population_log + GDP_pc_z + C(gni_class, Treatment(reference="H")) + C(gni_region, Treatment(reference="North America")) + country_cat_articles_log + views_baseline_log + view_country_article_log + oecd + in_code_lang + planed
________________________________________________________________________________________________
================= Fitting en =================
Estimated alpha as 15.15 (p=0.0000)
Deviance: 1635.62 | Null-deviance: 1868.23 | Pseudo R² (1 - D/D_0) = 0.1245
Pseudo ChiSq: 6983.82 | Good-Fit-ChiSq: 6865.17 | Residual DF: 6674
AIC: 154675.48 | BIC: -57156.43
H0 = Model provides adequate fit for data: p=1.00
Overdispersion factor: 1.05
--------------- Summary for en ---------------
                 Generalized Linear Model Regression Results                  
Dep. Variable:            views_7_sum   No. Observations:                 6695
Model:                            GLM   Df Residuals:                     6674
Model Family:  